In [105]:
# Imports

# Algorithms:
from koebe.algorithms.incrementalConvexHull import incrConvexHull, orientationPointE3, randomConvexHullE3
from koebe.algorithms.hypPacker import *
from koebe.algorithms.sampling import surfaceSampling, boundarySampling
from koebe.algorithms.poissonDiskSampling import slowAmbientSurfaceSampling, slowAmbientBoundarySampling
from koebe.algorithms.cvt import weightedCVT, worldToImgPixelCoords
from koebe.algorithms.tutteEmbeddings import tutteEmbeddingE2

# Graphics:
from koebe.graphics.spherical2viewer import *
from koebe.graphics.euclidean2viewer import PoincareDiskViewer, makeStyle, E2Viewer

# Geometries:
from koebe.geometries.orientedProjective2 import DiskOP2
from koebe.geometries.euclidean3 import PointE3, VectorE3, SegmentE3
from koebe.geometries.euclidean2 import SegmentE2, PointE2, VectorE2
from koebe.geometries.commonOps import inner_product31

# Linear Algebra:
import numpy as np
from numpy.linalg import matrix_rank, norm
from scipy.linalg import null_space

# Image creation:
from PIL import Image, ImageDraw, ImageFilter

# Other:
import random

In [106]:
poly = randomConvexHullE3(16)
poly.outerFace = poly.faces[0]

tutteGraph = tutteEmbeddingE2(poly)

In [107]:
 from koebe.graphics.euclidean2viewer import UnitScaleE2Sketch, makeStyle
viewer = UnitScaleE2Sketch()
#viewer.addAll([v.data for v in poly.verts])poly = randomConvexHullE3(16)

#for _ in range(60):
poly = randomConvexHullE3(16)
poly.outerFace = poly.faces[0]
tutteGraph = tutteEmbeddingE2(poly)
segments = [SegmentE2(e.aDart.origin.data, e.aDart.twin.origin.data) for e in tutteGraph.edges]
viewer.addAll(segments)
#viewer.pushAnimFrame()
#points = [v.data for v in tutteGraph.verts]
#viewer.addAll(points)

# for point in points:
#     viewer.setStyle(point, makeStyle(fill="#f00"))

for seg in segments:
    viewer.setStyle(seg, makeStyle(stroke="#000", strokeWeight=0.5))
viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=500, objects='[[{"type": "SegmentE2", "endpoints": [[0.45965048541981673, -0.01203757321922621…

In [108]:
dists = [(v.data - PointE3.O).normSq() for v in tutteGraph.verts]
closestToOriginIdx = dists.index(min(dists))
packing, _ = maximalPacking(
    tutteGraph, 
    num_passes=1000, 
    centerDartIdx = tutteGraph.darts.index(tutteGraph.verts[closestToOriginIdx].aDart)
)

In [109]:
flipEdgeIdx = 21
flipEdge = packing.edges[flipEdgeIdx]

# Run this to view the circle packing
viewer = E2Viewer(800, 800, scale = 1.0 / (4800*0.5-10))
blueStyle = makeStyle(stroke="#00f", strokeWeight=0.5)
redStyle = makeStyle(stroke="#f00", strokeWeight=2.5)
edgeSegs = [(SegmentE2(e.aDart.origin.data.toPoincareCircleE2().center, 
                      e.aDart.dest.data.toPoincareCircleE2().center), 
             blueStyle if e != flipEdge else redStyle)
            for e in packing.edges]
viewer.addAll([v for v in packing.verts])
viewer.addAll(edgeSegs)
viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=800, objects='[[{"type": "CircleE2", "center": [0.3064462058910917, 0.44013690287582424], "rad…

In [110]:
verts31 = [DiskOP2.fromCircleE2(v.data.toPoincareCircleE2()).toDiskS2() 
           for v in packing.verts]
verts31 += [DiskS2(0,0,0,0)] # Add in the origin.

# Labeling each vertex with its index in the vertex list is
# useful for building the rigidity matrix
for i in range(len(packing.verts)):
    packing.verts[i].name = i

edgeList = [(e.aDart.origin.name, e.aDart.dest.name) for e in packing.edges]
edgeList += [(vIdx, len(verts31) - 1) for vIdx in range(len(verts31) - 1)]

# Build the rigidity matrix. 
def edgeRow(edgeIdx, disks, edgeList):
    i, j = edgeList[edgeIdx]
    pi = tuple(disks[i])
    pj = tuple(disks[j])
    cij = [pj[0] - pi[0], pj[1] - pi[1], pj[2] - pi[2], pi[3] - pj[3]]
    cji = [pi[0] - pj[0], pi[1] - pj[1], pi[2] - pj[2], pj[3] - pi[3]]
    return np.array([
        cij if vIdx == i else
        cji if vIdx == j else
        [0,0,0,0]
        for vIdx in range(len(disks) - 1)
    ]).flatten()

def vertexPin(vIdx, disks):
    r1 = np.array([[1, 0, 0, 0] if i == vIdx else [0, 0, 0, 0] 
                   for i in range(len(disks) - 1)]).flatten()
    r2 = np.array([[0, 1, 0, 0] if i == vIdx else [0, 0, 0, 0] 
                   for i in range(len(disks) - 1)]).flatten()
    r3 = np.array([[0, 0, 1, 0] if i == vIdx else [0, 0, 0, 0] 
                   for i in range(len(disks) - 1)]).flatten()
    r4 = np.array([[0, 0, 0, 1] if i == vIdx else [0, 0, 0, 0] 
                   for i in range(len(disks) - 1)]).flatten()
    return np.array([r1, r2, r3, r4])

def rigidityMatrix(disks, edgeList, pinSet = None):
    edgeRows = np.array([
        edgeRow(eIdx, disks, edgeList) 
        for eIdx in range(len(edgeList)) if eIdx != flipEdgeIdx
    ])
    if pinSet is None:
        pinSet = [0, 1, 2]
    pinRows0    = vertexPin(pinSet[0], disks)
    pinRows1    = vertexPin(pinSet[1], disks)
    pinRows2    = vertexPin(pinSet[2], disks)
    pinRowsOrig = vertexPin(len(disks) - 1, disks)
    return np.concatenate((edgeRows, pinRows0, pinRows1, pinRows2, pinRowsOrig))

In [111]:
def newtonStepOneDisk(disk, ns, deltaT):
    a = disk.a + deltaT * ns[0]
    b = disk.b + deltaT * ns[1]
    c = disk.c + deltaT * ns[2]
    d = disk.d + deltaT * ns[3]
    inv31 = 1.0# / inner_product31(a, b, c, d, a, b, c, d)
    return DiskS2(a * inv31, b * inv31, c * inv31, d * inv31)

def newtonStep(disks, edgeList, deltaT, prevNs, pinSet = None, direction = 1):
    R = rigidityMatrix(disks, edgeList, pinSet)
    ns = null_space(R).reshape(len(disks)-1, 4)
    if prevNs is not None:
        posDot = np.dot(np.transpose(ns.reshape(64, 1)), prevNs.reshape(64, 1))[0][0]
        negDot = np.dot(np.transpose((-ns).reshape(64, 1)), prevNs.reshape(64, 1))[0][0]
        if negDot > posDot:
            ns = -ns
    else:
        ns = ns * direction
    return [newtonStepOneDisk(disks[vIdx], ns[vIdx], deltaT) 
            for vIdx in range(len(disks) - 1)] + [disks[-1]], ns

def newtonAnimation(initialDisks, edgeList, iterationCount, deltaT, pinSet = None, direction = 1):
    animation = [initialDisks]
    prevNs = None
    for _ in range(iterationCount):
        nextFrame, prevNs = newtonStep(animation[-1], edgeList, deltaT, prevNs, pinSet, direction)
        animation.append(nextFrame)
    return animation

In [112]:
anim = newtonAnimation(verts31, edgeList, 1600,0.02, pinSet = [0,1,2], direction = -1)


In [126]:


# Run this to view the circle packing
viewer = E2Viewer(800, 800, scale = 1/400)
viewer.addAll([v.sgProjectToOP2() for v in verts31[0:len(verts31)-1]])

viewer.pushAnimFrame()

for i in range(800):#len(anim)):
    disksForward = [(d.sgProjectToOP2(), blueStyle) 
            for d in anim[i][:-1]]
    viewer.pushAnimFrame()
    viewer.addAll(disksForward)

# disksForward = [(d.sgProjectToOP2(), blueStyle) 
#             for d in anim[1][:-1]]
# viewer.addAll(disksForward)

#viewer.show()
print(verts31[0])

<IPython.core.display.Javascript object>

DiskS2(a=0.9273774748250564, b=-0.8802738057516485, c=-0.6128924117821835, d=-1.0726225251749437)


In [127]:
# Run this to view the circle packing
viewer = S2Viewer(800, 800)

diskSet = [v for v in verts31[0:len(verts31)-1]]
viewer.addAll(diskSet)
for d in diskSet:
    viewer.setStyle(d, blueStyle)
    
n = 65
redStyle = makeStyle(stroke = "#f00", strokeWeight = 0.5)
for i in range(n):#len(anim)):
    disksForward = [d for d in anim[i][:-1]]
    orthos = [CPlaneS2.throughThreeDiskS2(*[disksForward[v.name] 
                                            for v in f.vertices()]) 
              for f in packing.faces]
    viewer.pushAnimFrame()
    viewer.addAll(disksForward)
    #viewer.addAll(orthos)
    for d in disksForward:
        viewer.setStyle(d, blueStyle)
    for o in orthos:
        viewer.setStyle(o, redStyle)
        

for i in range(n):#len(anim)):
    disksForward = [d for d in anim[n - i - 1][:-1]]
    orthos = [CPlaneS2.throughThreeDiskS2(*[disksForward[v.name] 
                                            for v in f.vertices()]) 
              for f in packing.faces]
    viewer.pushAnimFrame()
    viewer.addAll(disksForward)
    #viewer.addAll(orthos)
    for d in disksForward:
        viewer.setStyle(d, blueStyle)
    for o in orthos:
        viewer.setStyle(o, redStyle)


<IPython.core.display.Javascript object>

In [128]:

viewer.show()

S2Sketch(height=800, objects='[[{"type": "DiskS2", "disk": [0.9273774748250564, -0.8802738057516485, -0.612892…

In [121]:
[v.name for v in packing.faces[1].vertices()]

[5, 4, 2]

# Question on 4-gon flexibility

If we have a circle 4-gon with one tangency and all else disjoint does the flow always create another tangency?

In [94]:
# Run this to view the circle packing
construction = S2Viewer(800, 800)

x  = PointS2(1, 0, 0)
y  = PointS2(0, 1, 0)
z  = PointS2(0, 0, 1)

p1 = PointS2(1, -0.5, 0)
p2 = PointS2(1, 0.5, 0)
p3 = PointS2(0, 1, 0)
p4 = PointS2(0.5, 0.3, 0.7)
p5 = PointS2(0, 1, 1)

A = DiskS2.withDiameterS2(p1, p2)
B = DiskS2.withDiameterS2(p2, p3)
C = DiskS2.withCenterAndRadiusS2(p4, math.pi/16)
D = DiskS2.withCenterAndRadiusS2(p5, math.pi/18)

construction.setStyle(C, redStyle)
construction.setStyle(D, blueStyle)

construction.addAll([p1, p2, p3, p4, p5, A, B, C, D])
construction.show()

<IPython.core.display.Javascript object>

S2Sketch(height=800, objects='[[{"type": "PointE3", "point": [0.8944271909999159, -0.4472135954999579, 0.0]}, …

In [87]:
DiskS2.withCenterAndRadiusS2 = withCenterAndRadiusS2